<a href="https://colab.research.google.com/github/Adhvik09-oss/Adhvik09-oss/blob/main/SnapCareipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
#importing out neccessary modules
import torch as nn
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision
import torch
import torchvision.models as models
import torch.optim as optim


In [10]:
#writing our device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

### for more information on the different transfer learning architectures look at:
https://docs.pytorch.org/vision/0.9/models.html

In [11]:
resnet = models.resnet50(pretrained=True)

In [12]:
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [13]:
import torch.nn as nn
# we have to change the final layer because our dataset(SD-198) has 198 classes, unlike the standard 100 that resnet-50 comes with
resnet.fc = nn.Linear(in_features=2048, out_features=198)

In [14]:
#freeze all the layers so we won't have to train it fully
for param in resnet.parameters():
  param.requires_grad = False

#unfreeze the fully connected layer
for param in resnet.fc.parameters():
  param.requires_grad = True

In [15]:
#selecting our loss function and optimizers
loss_fn = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(params = resnet.parameters(),
                            lr = 0.01,
                            momentum = 0.09)

In [17]:
#importing our dataset and preparing to create a dataloader
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
import zipfile
import os


skin_data_path = '/content/drive/MyDrive/data.zip' #this is the google drive path of the dataset
extract_path = '/content/SD-198'  # Local Colab path to unzip

with zipfile.ZipFile(skin_data_path, 'r') as zip_ref: #this simply just extracts the zip file with data to a certain path
    zip_ref.extractall(extract_path)

In [19]:
import os
#double checking the strucure of our data

print(os.listdir(extract_path))  # Should list class folders
sample_class = os.listdir(extract_path)[0]
print("Sample images in class:", sample_class)
print(os.listdir(os.path.join(extract_path, sample_class))[:5])


['Racquet_Nail', 'Scarring_Alopecia', 'Drug_Eruption', 'Cellulitis', 'Basal_Cell_Carcinoma', 'Fibroma', 'Tinea_Corporis', 'Nail_Dystrophy', 'Androgenetic_Alopecia', 'Eccrine_Poroma', 'Granuloma_Annulare', 'Nevus_Spilus', 'Halo_Nevus', 'Pyoderma_Gangrenosum', 'Beckers_Nevus', 'Onychoschizia', 'Blue_Nevus', 'Discoid_Lupus_Erythematosus', 'Actinic_solar_Damage(Actinic_Cheilitis)', 'Pityriasis_Alba', 'Darier-White_Disease', 'Tinea_Faciale', 'Seborrheic_Keratosis', 'Geographic_Tongue', 'Onychomycosis', 'Beaus_Lines', 'Follicular_Retention_Cyst', 'Psoriasis', 'Xerosis', 'Actinic_solar_Damage(Actinic_Keratosis)', 'Stasis_Dermatitis', 'Crowes_Sign', 'Callus', 'Acne_Vulgaris', 'Paronychia', 'Cutanea_Larva_Migrans', 'Eczema', 'Hidradenitis_Suppurativa', 'Impetigo', 'Dyshidrosiform_Eczema', 'Mal_Perforans', 'Keratoacanthoma', 'Terrys_Nails', 'Ichthyosis', 'Actinic_solar_Damage(Solar_Purpura)', 'Lymphangioma_Circumscriptum', 'Ganglion', 'Erythema_Ab_Igne', 'Neurodermatitis', 'Junction_Nevus', 'Fol

In [30]:
from torchvision import transforms
from torch.utils.data import DataLoader,random_split # import random split for later use
from torchvision.datasets import ImageFolder

train_data = transforms.Compose([
    transforms.Resize((224,224)), #resizes img to 224x224 which is what resnet uses
    transforms.ToTensor(), #transforms from PIL to Pytorch Tensor
    transforms.Normalize([0.485,0.456,0.406],
                          [0.229, 0.224, 0.225])

])

dataset = ImageFolder(root = extract_path, transform = train_data) # this is our data in the form of a Image Folder, we will have to change this into a

train_split = int(0.8*len(dataset))
testing_split = len(dataset)-train_split

train_dataset,test_dataset = random_split(dataset,[train_split,testing_split])

train_loader = DataLoader(train_dataset,batch_size = 32, shuffle = True)
test_loader = DataLoader(test_dataset,batch_size = 32, shuffle = False)

In [33]:
#using our premade accuracy_function
def accuracy(y_true,y_pred):
  correct = torch.eq(y_true,y_pred).sum().item()#gives amount of correct preds in the form of a python int
  acc = (correct/len(y_pred))*100#divdes it by total num of predictions to return a accuracy
  return acc


In [34]:
#importing a function previously made to measure the time
def print_train_time(start:float,
                     end:float,
                     device: torch.device = None):

  "print difference between start and end time"
  total_time = end-start
  print(f"train time on {device}: {total_time:.3f} seconds")

In [ ]:
#importing our previously made model evalutaion function:
from tqdm.auto import tqdm
def eval_model(model:torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               accuracy_fn):
  '''returns a dictionary contatining hte results of model predicting on data_loader'''
  loss,acc = 0,0
  model.eval()
  with torch.inference_mode():
    for X,y in tqdm(data_loader):
      #make predictions
      y_pred = model(X)
      #Accumulate the loss and the acc values per batch
      loss+= loss_fn(y_pred,y)
      acc+= accuracy_fn(y_true=y,
                        y_pred = y_pred.argmax(dim=1)) #raw ouputs are logits accuracy only takes true labels so we need to label it
    #Scale loss and acc to find avg loss/acc per batch
    loss/=len(data_loader)
    acc/= len(data_loader)
    return {
            "model_loss":loss.item(),
            "model_acc":acc}


In [ ]:
#importing our previously made training and testing loop functions:
def train_step(model: torch.nn.Module(),
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module(),
               optimizer: torch.optim.Optimizer,
               accuracy,
               device: torch.device = device):
  pass
  train_loss,train_acc = 0,0
  for batch, (X,y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    #do the forward pass
    y_pred = model(X)
    #calculate loss
    loss = loss_fn(y_pred,y)
    #accumulate loss
    train_loss += loss
    train_acc += accuracy(y_true=y,
                                 y_pred=y_pred.argmax(dim=1)) # Go from logits -> pred labels
    optimizer.zero_grad()
    #loss backward
    loss.backward()
    #optimizer zero
    optimizer.step()
    #print what's happening

  #divide total training loss over len
  train_loss /= len(dataloader)
  train_acc /= len(dataloader)

  print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")

def test_step(model: torch.nn.Module(),
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module(),
              optimizer: torch.optim.Optimizer,
              accuracy,
              device: torch.device = device):
  test_acc,test_loss = 0,0
  model.eval()

  with torch.inference_mode():
    for X,y in dataloader:
      X, y = X.to(device), y.to(device)
      #forward pass
      test_pred = model(X) #raw logits

    # 2. Calculate loss and accuracy
      test_loss += loss_fn(test_pred, y)
      test_acc += accuracy(y_true=y,
                  y_pred=test_pred.argmax(dim=1) # Go from logits -> pred labels
              )
    test_loss/=len(dataloader)
    test_acc/=len(dataloader)
    print(f"\n train loss: {train_loss} Test loss {test_loss}, test_acc {test_acc}")